In [2]:
import pandas as pd
import numpy as np
import os
import scipy.io

# Importing Dataset

In [3]:
filename = os.path.join( os.getcwd(), '../..', 'data/original/deidentified_03-22-2021.csv' )
data = pd.read_csv(filename)

In [4]:
data.rename( columns={'Unnamed: 0':'ID'}, inplace=True )
pd.options.display.max_columns = None

## Looking at missing values

In [39]:
data.isna().sum()[data.isna().sum()!=0]

HASHYPERTENSION         1
HASTRANSPLANT           1
MI                      1
CHF                     1
PVD                     1
STROKE                  1
PUD                     1
LIVERDISEASE            1
RHEUM                   1
DIABETES                1
COPD                    1
KIDNEYDISEASE           1
CANCER                  1
DEMENTIA                1
HEMIPLEGIA              1
averageBMI           3859
glucoseMedian        8497
glucoseLast          8497
SodiumMedian         8508
SodiumLast           8508
ALTMedian            8872
ALTLast              8872
plateletsMedian      8452
plateletsLast        8452
GFRMedian            8518
GFRLast              8518
CreatinineMedian     8509
CreatinineLast       8509
TroponinMedian      13475
TroponinLast        13475
ddimerMedian        16049
ddimerLast          16049
systolicMedian      11484
systolicLast        11484
diastolicMedian     11499
diastolicLast       11499
dtype: int64

## Drop features with more than 50% missing values

In [47]:
#Drop features with more than 10000 missing values
data.drop(data.isna().sum()[data.isna().sum()>10000].index,axis=1, inplace = True)

In [48]:
features = data[data.columns[1:-11]].copy()
labels = data[data.columns[-11:]].copy()

## Identify Features vs Target Labels

In [49]:
display(features)
display(labels.head())

,medCat__5HT3_receptor_antagonists,anyNarcotic,anyNSAID,anyBenzo,medCat__CNS_stimulants,medCat__SSNRI_antidepressants,medCat__SSRI_antidepressants,medCat__analgesic_combinations,medCat__anorexiants,medCat__anticonvulsants,medCat__antidepressants,medCat__antihistamines,medCat__coumarins_and_indanediones,medCat__factor_Xa_inhibitors,medCat__gamma-aminobutyric_acid_analogs,medCat__miscellaneous_analgesics,medCat__miscellaneous_anticonvulsants,medCat__miscellaneous_antidepressants,medCat__miscellaneous_antiemetics,"medCat__miscellaneous_anxiolytics,_sedatives_and_hypnotics",medCat__phenothiazine_antiemetics,medCat__phenothiazine_antipsychotics,medCat__phenylpiperazine_antidepressants,medCat__proton_pump_inhibitors,medCat__skeletal_muscle_relaxants,medCat__smoking_cessation_agents,AGEINYEARS,HASHYPERTENSION,HASTRANSPLANT,MI,CHF,PVD,STROKE,PUD,LIVERDISEASE,RHEUM,DIABETES,COPD,KIDNEYDISEASE,CANCER,HIV,DEMENTIA,HEMIPLEGIA,AfricanAmerican,Asian,Caucasian,Hispanic,Male,averageBMI,glucoseMedian,glucoseLast,SodiumMedian,SodiumLast,ALTMedian,ALTLast,plateletsMedian,plateletsLast,GFRMedian,GFRLast,CreatinineMedian,CreatinineLast,countPriorER,countPriorAdmits
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,90,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,0,1.0,0.0,1,0,0,0,0,24.369050,94.5,88.0,141.0,143.0,7.0,7.0,120.0,256.0,48.5,24.0,1.045,1.87,2,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,NaN,74.0,75.0,138.5,141.0,NaN,NaN,277.0,260.0,57.5,66.0,0.935,0.82,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,1.0,0.0,1.0,1.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,3.0,2.0,0,1.0,0.0,1,0,0,0,0,33.198392,87.0,87.0,166.0,166.0,7.0,7.0,74.0,74.0,12.0,12.0,3.480,3.48,1,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,NaN,114.0,147.0,137.0,137.0,8.0,7.0,228.0,228.0,66.0,66.0,0.820,0.82,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17801,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
17802,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
17803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,1,0,0,0,0,39.762802,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
17804,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


,death30,death60,death90,ICU,vent,anyCatastrophic,Admit30Days,Admit60Days,Admit90Days,Admit7Days,Admit14Days
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,1,1,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0


In [10]:
#Show unique values in each features to figure out categorical features vs continous features
for col in features:
    print (col)
    print(features[col].unique())

medCat__5HT3_receptor_antagonists
[0 1]
anyNarcotic
[0 1]
anyNSAID
[0 1]
anyBenzo
[0 1]
medCat__CNS_stimulants
[0 1]
medCat__SSNRI_antidepressants
[0 1]
medCat__SSRI_antidepressants
[0 1]
medCat__analgesic_combinations
[0 1]
medCat__anorexiants
[0 1]
medCat__anticonvulsants
[0 1]
medCat__antidepressants
[0 1]
medCat__antihistamines
[0 1]
medCat__coumarins_and_indanediones
[0 1]
medCat__factor_Xa_inhibitors
[1 0]
medCat__gamma-aminobutyric_acid_analogs
[0 1]
medCat__miscellaneous_analgesics
[0 1]
medCat__miscellaneous_anticonvulsants
[0 1]
medCat__miscellaneous_antidepressants
[0 1]
medCat__miscellaneous_antiemetics
[0 1]
medCat__miscellaneous_anxiolytics,_sedatives_and_hypnotics
[0 1]
medCat__phenothiazine_antiemetics
[0 1]
medCat__phenothiazine_antipsychotics
[0 1]
medCat__phenylpiperazine_antidepressants
[0 1]
medCat__proton_pump_inhibitors
[0 1]
medCat__skeletal_muscle_relaxants
[0 1]
medCat__smoking_cessation_agents
[0 1]
AGEINYEARS
[90 89 88 87 86 85 84 83 82 81 80 79 78 77 76 75 

## One-hot encode non-binary categorical features

In [11]:
categorical_features = ['LIVERDISEASE','DIABETES','KIDNEYDISEASE','CANCER','HIV','HEMIPLEGIA']
for category in categorical_features:
    one_hot = pd.get_dummies(features[category], prefix=category)
    features = features.drop(category,axis = 1)
    # Join the encoded df
    features = features.join(one_hot)
print(one_hot.head())
print(features.head())

   HEMIPLEGIA_0.0  HEMIPLEGIA_1.0  HEMIPLEGIA_2.0
0               1               0               0
1               1               0               0
2               1               0               0
3               1               0               0
4               1               0               0
   medCat__5HT3_receptor_antagonists  anyNarcotic  anyNSAID  anyBenzo  \
0                                  0            0         0         0   
1                                  0            0         0         0   
2                                  0            0         0         0   
3                                  0            0         0         0   
4                                  0            0         0         0   

   medCat__CNS_stimulants  medCat__SSNRI_antidepressants  \
0                       0                              0   
1                       0                              0   
2                       0                              0   
3                    

In [12]:
#Show unique values in each features to figure out categorical features vs continous features
for col in features:
    print (col)
    print(len(features[col].unique()))

medCat__5HT3_receptor_antagonists
2
anyNarcotic
2
anyNSAID
2
anyBenzo
2
medCat__CNS_stimulants
2
medCat__SSNRI_antidepressants
2
medCat__SSRI_antidepressants
2
medCat__analgesic_combinations
2
medCat__anorexiants
2
medCat__anticonvulsants
2
medCat__antidepressants
2
medCat__antihistamines
2
medCat__coumarins_and_indanediones
2
medCat__factor_Xa_inhibitors
2
medCat__gamma-aminobutyric_acid_analogs
2
medCat__miscellaneous_analgesics
2
medCat__miscellaneous_anticonvulsants
2
medCat__miscellaneous_antidepressants
2
medCat__miscellaneous_antiemetics
2
medCat__miscellaneous_anxiolytics,_sedatives_and_hypnotics
2
medCat__phenothiazine_antiemetics
2
medCat__phenothiazine_antipsychotics
2
medCat__phenylpiperazine_antidepressants
2
medCat__proton_pump_inhibitors
2
medCat__skeletal_muscle_relaxants
2
medCat__smoking_cessation_agents
2
AGEINYEARS
89
HASHYPERTENSION
3
HASTRANSPLANT
3
MI
3
CHF
3
PVD
3
STROKE
3
PUD
3
RHEUM
3
COPD
3
DEMENTIA
3
AfricanAmerican
2
Asian
2
Caucasian
2
Hispanic
2
Male
2
av

# Split Dataset into Training and Testing Set 

In [13]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size = 0.2)

In [14]:
path = os.path.join( os.getcwd(), '../..', 'data/preprocessed_BMED8813/' )
filename = path+'TrainFeatures.csv'
X_train.to_csv(r''+filename,index=False)
filename = path+'TrainLabels.csv'
y_train.to_csv(r''+filename,index=False)
filename = path+'TestFeatures.csv'
X_test.to_csv(r''+filename,index=False)
filename = path+'TestLabels.csv'
y_test.to_csv(r''+filename,index=False)

In [15]:
print("Training\n",len(y_train))
print("Testing\n",len(y_test))

Training
 14244
Testing
 3562
